In [21]:
import os
os.chdir("/gpfs/home/asun/jin_lab/yap/pipeline0_bt2_local_alignment/split_r")
print(os.getcwd())

/gpfs/group/jin/asun/yap/pipeline0_bt2_local_alignment/split_r


In [22]:
%%bash
samtools view intersect.bam | cut -f1 | sort -u > readnames.txt

module load seqtk
seqtk subseq /gpfs/home/asun/jin_lab/yap/raw_data/R/TEST-R-1-A1_ii1_L003_R1_ii2.fq.gz readnames.txt > grna_r1.fq
seqtk subseq /gpfs/home/asun/jin_lab/yap/raw_data/R/TEST-R-1-A1_ii1_L003_R2_ii2.fq.gz readnames.txt > grna_r2.fq

In [23]:
import pandas as pd
import subprocess
from Bio.Seq import Seq

# Function to reverse complement if RC column is "RC"
def conditional_reverse_complement(row):
    if row["RC"] == "RC":
        return str(Seq(row["SEQ"]).reverse_complement())
    return row["SEQ"]  # Keep the sequence unchanged if not "RC"

# Load BAM into DataFrame
sam_output = subprocess.run(["samtools", "view", "intersect.bam"], capture_output=True, text=True)
sam_lines = sam_output.stdout.strip().split("\n")

# Convert SAM to DataFrame
columns = ["QNAME", "FLAG", "RNAME", "POS", "MAPQ", "CIGAR", "RNEXT", "PNEXT", "TLEN", "SEQ", "QUAL"]
data = [line.split("\t")[:11] for line in sam_lines]
df = pd.DataFrame(data, columns=columns)

# Convert FLAG to numeric
df["FLAG"] = pd.to_numeric(df["FLAG"])
df["RC"] = df["FLAG"].apply(lambda x: "RC" if x & 16 else "FORWARD")

rp_indices = pd.read_csv("/gpfs/home/asun/jin_lab/yap/raw_data/384RPIndexes.csv")
rp_indices = rp_indices.rename(columns={"RP Index": "RP_INDEX", "Position": "WELL"})   

# Apply the function to create the TRUE_SEQ column
df["TRUE_SEQ"] = df.apply(conditional_reverse_complement, axis=1)

df["RP_INDEX"] = df["TRUE_SEQ"].apply(lambda x: x[:8])

# Assign read type
df["READ_TYPE"] = df["FLAG"].apply(lambda x: "READ_1" if x & 64 else ("READ_2" if x & 128 else "UNPAIRED"))

df["READ_1"] = ""

readnames = df.loc[(df["READ_TYPE"] == "READ_2"), "QNAME"].tolist()

with open("grna_r1.fq", "r") as fq:
    while True:
        header = fq.readline().strip()  
        seq = fq.readline().strip()     
        fq.readline()                   
        fq.readline()                   
        
        if not header:
            break  # End of file
        
        qname = header.lstrip("@")

        if qname in df["QNAME"].values:
            first_8_bp = seq[:8]  # Extract the first 8 bases
            df.loc[df["QNAME"] == qname, "READ_1"] = seq
            df.loc[df["QNAME"] == qname, "RP_INDEX"] = first_8_bp


df = df.merge(rp_indices, on=["RP_INDEX"], how="left")
df


# Filtering condition
filtered_df = df[(df["RNEXT"] != "=") | (df["READ_TYPE"] == "READ_1")]
len(filtered_df)

# Save filtered reads
#filtered_df.to_csv("filtered_reads.sam", sep="\t", index=False, header=False)

# Save output
#df.to_csv("reads_with_type.sam", sep="\t", index=False, header=False)

3

In [24]:
df_filtered = df.drop_duplicates(subset='QNAME', keep='first')
len(df_filtered)

5

In [25]:
df_filtered

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,RC,TRUE_SEQ,RP_INDEX,READ_TYPE,READ_1,WELL
0,lh00134:653:22MKYCLT4:3:1102:36984:8090,83,Foxg1_g1,2,9,57S72M21S,=,2,185,AGTGACTTGAATTCAGACGTGTGTTCTTCCGATCTTAGGATGGAGG...,99*99*9*99**9*9*******9****9*I9***9**II*9I*I*9...,RC,TCAAGCAATAAGGCAGAGTACCAAGTTGATAACGGACTAGCCTTAT...,TCAAGCAA,READ_1,TCAAGCAATAAGGCAGAGTACCAAGTTGATAACGGACTAGCCTTAT...,NaN
2,lh00134:653:22MKYCLT4:3:1133:49176:26320,153,Safe_g1,1,9,50S72M28S,=,1,0,AGTATTTCGATTTCTTGACTTTATATATCTTGTGGAAAGGACGAAA...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,RC,GGGAGAGAAAAGATTAATGTAGAGTATTAAGTTGATAATGGATTAG...,GATACCTG,READ_2,GATACCTGAATACATCTCACTATAGGGAAATACGATCCGGAGGGCC...,L21
3,lh00134:653:22MKYCLT4:3:1196:37979:7011,153,Foxg1_g1,1,2,72S14M2D53M11S,=,1,0,ATATGCTTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATA...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,RC,GGAGAAAAAATTTGATAATGGATTAGTTTTATTTTAACTTGCTATT...,CGATGTAA,READ_2,CGATGTAATAAGGCTGTTAGAGATAATTGGAATTAATTTGACTGTA...,NaN
4,lh00134:653:22MKYCLT4:3:1198:47202:21472,99,Foxg1_g1,1,22,100S50M,=,1,191,TCAAGCAATAAGGCCGCTATCAACTTAATAAGCAGTGGTATCAAGG...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,FORWARD,TCAAGCAATAAGGCCGCTATCAACTTAATAAGCAGTGGTATCAAGG...,TCAAGCAA,READ_1,TCAAGCAATAAGGCCGCTATCAACTTAATAAGCAGTGGTATCAAGG...,NaN
6,lh00134:653:22MKYCLT4:3:2496:1574:20491,83,Foxg1_g1,2,9,57S72M21S,=,2,185,AGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTGAGGATGAATG...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,RC,TCAAGCAATAAGGCAGAGTACCAAGTTGATAACGGACTAGCCTTAT...,TCAAGCAA,READ_1,TCAAGCAATAAGGCAGAGTACCAAGTTGATAACGGACTAGCCTTAT...,NaN


In [26]:
final_df = df_filtered[["QNAME", "CIGAR", "RNAME", "MAPQ", "READ_TYPE", "WELL"]]
final_df
final_df.to_csv("r.tsv", sep="\t", index=False, header=True)

In [45]:
#test = df[(df["RNEXT"] != "=")]
test = filtered_df[(filtered_df["READ_TYPE"] == "READ_2")]
test

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,READ_TYPE,RP_INDEX,WELL_x,RC,TRUE_SEQ,WELL_y
0,lh00134:653:22MKYCLT4:3:1102:23021:5428,177,Foxg1_g1,1,22,66S72M12S,chr8,30412233,0,TTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIIIIIIII...,READ_2,GAGGGAAA,NaN,RC,GAGGGAAATATTAAGTTGATAATGGATTAGTTTTATTTTAATTTGC...,NaN
56,lh00134:653:22MKYCLT4:3:1338:39929:15208,129,Safe_g2,1,12,75S75M,chr10,17033755,0,ATGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGG...,99IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIII...,READ_2,ATGATGGG,NaN,FORWARD,ATGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGG...,NaN
57,lh00134:653:22MKYCLT4:3:1340:3451:1799,161,Safe_g2,1,12,76S74M,chr9,74596377,0,GGAGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGG...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,GGAGATGG,NaN,FORWARD,GGAGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGG...,NaN
80,lh00134:653:22MKYCLT4:3:1431:7116:8118,177,Foxg1_g1,1,9,48S73M29S,chr8,110630552,0,CAGAGTGAATGGGGGCTTTATATATCTTGTGGAAAGGACGAAACAC...,IIII99IIII9IIII*I9IIIIIII9IIIIIII9IIIIIII9III9...,READ_2,GGAGGCGC,NaN,RC,GGAGGCGCACCGACTCGGTGCCACTTTTTCAAGTTGATAACGGACT...,NaN
90,lh00134:653:22MKYCLT4:3:1494:23361:11033,129,Safe_g2,1,9,70S80M,chr16,82949479,0,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,GGGGTGTG,NaN,FORWARD,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,NaN
92,lh00134:653:22MKYCLT4:3:1494:23377:11061,129,Safe_g2,1,9,70S80M,chr6,58561537,0,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,IIII9I9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIII...,READ_2,GGGGTGTG,NaN,FORWARD,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,NaN
108,lh00134:653:22MKYCLT4:3:2196:44896:5904,177,Safe_g1,1,22,66S72M12S,chr13,69470591,0,CTTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,GGAGGAGA,NaN,RC,GGAGGAGATATTAAGTTGATAATGGATTAGTTTTATTTTAACTTGC...,NaN
109,lh00134:653:22MKYCLT4:3:2196:44904:5918,145,Safe_g1,1,22,66S72M12S,chr18,35519142,0,CTTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,GGAGGAGA,NaN,RC,GGAGGAGATATTAAGTTGATAATGGATTAGTTTTATTTTAACTTGC...,NaN
180,lh00134:653:22MKYCLT4:3:2385:18499:23602,145,Foxg1_g1,2,9,66S71M13S,chr9,116569877,0,CTTACCGTAACTGGAAAGTATTTCGATTTCTTGGCTTTATATATCT...,IIII9IIIIIII*IIII9II9I9IIIIIIIIIII9IIIIIII9I9I...,READ_2,GGGGAAGA,NaN,RC,GGGGAAGAGTATTAAGTTGATAATGGATTAGTTTTATTTTAACTTG...,NaN


In [51]:
result = test.loc[(test["READ_TYPE"] == "READ_2"), "QNAME"]

with open("r2_readnames.txt", "w") as f:
    for item in result.tolist():
        f.write(str(item) + "\n")

In [52]:
%%bash

grep -F -f r2_readnames.txt grna_r1.fq -A 3 > matched_r1.fq

In [55]:
test = test.merge(rp_indices, on=["RP_INDEX"], how="left")
test

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,READ_TYPE,RP_INDEX,WELL_x,RC,TRUE_SEQ,WELL_y,READ_1,WELL
0,lh00134:653:22MKYCLT4:3:1102:23021:5428,177,Foxg1_g1,1,22,66S72M12S,chr8,30412233,0,TTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIIIIIIII...,READ_2,ACAACAGC,NaN,RC,GAGGGAAATATTAAGTTGATAATGGATTAGTTTTATTTTAATTTGC...,NaN,ACAACAGCATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTACATCTA...,J17
1,lh00134:653:22MKYCLT4:3:1338:39929:15208,129,Safe_g2,1,12,75S75M,chr10,17033755,0,ATGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGG...,99IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIII...,READ_2,CTTACAGC,NaN,FORWARD,ATGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGG...,NaN,CTTACAGCATTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTACTTTT...,I22
2,lh00134:653:22MKYCLT4:3:1340:3451:1799,161,Safe_g2,1,12,76S74M,chr9,74596377,0,GGAGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGG...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,ACGAGAAC,NaN,FORWARD,GGAGATGGGTGTCGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGG...,NaN,ACGAGAACCTTTTTTTCTTTTTTTTTTTTTTTTTTTTTTTTCCGGC...,O9
3,lh00134:653:22MKYCLT4:3:1431:7116:8118,177,Foxg1_g1,1,9,48S73M29S,chr8,110630552,0,CAGAGTGAATGGGGGCTTTATATATCTTGTGGAAAGGACGAAACAC...,IIII99IIII9IIII*I9IIIIIII9IIIIIII9IIIIIII9III9...,READ_2,CATAGTAA,NaN,RC,GGAGGCGCACCGACTCGGTGCCACTTTTTCAAGTTGATAACGGACT...,NaN,CATAGTAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCG...,NaN
4,lh00134:653:22MKYCLT4:3:1494:23361:11033,129,Safe_g2,1,9,70S80M,chr16,82949479,0,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,ACGAGAAC,NaN,FORWARD,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,NaN,ACGAGAACTTTTTTATTTTTTTTTTTTTTTTTTTTTCTTTTTTTTT...,O9
5,lh00134:653:22MKYCLT4:3:1494:23377:11061,129,Safe_g2,1,9,70S80M,chr6,58561537,0,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,IIII9I9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIII...,READ_2,ACGAGAAC,NaN,FORWARD,GGGGTGTGGGGCAGTGGTAGAATTCTCGCCTCCCACGCGGGAGACC...,NaN,ACGAGAACTTTTTTATTTTTTTTTTTTTTTTTTTTTCTTTTTTTTT...,O9
6,lh00134:653:22MKYCLT4:3:2196:44896:5904,177,Safe_g1,1,22,66S72M12S,chr13,69470591,0,CTTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,GACGTCAT,NaN,RC,GGAGGAGATATTAAGTTGATAATGGATTAGTTTTATTTTAACTTGC...,NaN,GACGTCATTTTTTATTTTTTTTTTTTTTTTTTTTTTTTTGTAAGCA...,I17
7,lh00134:653:22MKYCLT4:3:2196:44904:5918,145,Safe_g1,1,22,66S72M12S,chr18,35519142,0,CTTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,READ_2,GACGTCAT,NaN,RC,GGAGGAGATATTAAGTTGATAATGGATTAGTTTTATTTTAACTTGC...,NaN,GACGTCATTTTTTATTTTTTTTTTTTTTTTTTTTTTTTTTAAGCAG...,I17
8,lh00134:653:22MKYCLT4:3:2385:18499:23602,145,Foxg1_g1,2,9,66S71M13S,chr9,116569877,0,CTTACCGTAACTGGAAAGTATTTCGATTTCTTGGCTTTATATATCT...,IIII9IIIIIII*IIII9II9I9IIIIIIIIIII9IIIIIII9I9I...,READ_2,CCAACAGC,NaN,RC,GGGGAAGAGTATTAAGTTGATAATGGATTAGTTTTATTTTAACTTG...,NaN,CCAACAGCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTACCCCTCC...,NaN


In [56]:
test.to_csv("r2_with_well.tsv", sep="\t", index=False, header=True)

In [ ]:
# Filter rows where column 13 is not NaN and column 11 equals "READ_1"
result = filtered_df.loc[filtered_df["WELL_y"].notna() & (filtered_df["READ_TYPE"] == "READ_1"), "QNAME"]

result.tolist()

In [37]:
final_df = filtered_df[["RNAME", "WELL_y"]]
final_df.to_csv("local_align_extra.tsv", sep="\t", index=False, header=True)

In [29]:
# Count rows with NaN in the 'POSITION' column
nan_count = filtered_df["WELL_y"].isna().sum()

print("Number of rows with NaN in 'WELL_y':", nan_count)

Number of rows with NaN in 'WELL_y': 18


In [30]:
# Count rows with NaN in the 'POSITION' column
nan_count = filtered_df["WELL_x"].isna().sum()

print("Number of rows with NaN in 'WELL_x':", nan_count)

Number of rows with NaN in 'WELL_x': 73


In [31]:
unique_count = filtered_df["QNAME"].nunique()
print(f"Number of unique QNAME values: {unique_count}")

Number of unique QNAME values: 99


In [34]:
# Filter rows where column 13 is not NaN and column 11 equals "READ_1"
result = filtered_df.loc[filtered_df["WELL_y"].notna() & (filtered_df["READ_TYPE"] == "READ_1"), "QNAME"]

result.tolist()

['lh00134:653:22MKYCLT4:3:1107:31103:16231',
 'lh00134:653:22MKYCLT4:3:1121:13337:8917',
 'lh00134:653:22MKYCLT4:3:1130:28635:15180',
 'lh00134:653:22MKYCLT4:3:1132:8677:2359',
 'lh00134:653:22MKYCLT4:3:1133:3257:17408',
 'lh00134:653:22MKYCLT4:3:1136:30067:17913',
 'lh00134:653:22MKYCLT4:3:1152:6331:18333',
 'lh00134:653:22MKYCLT4:3:1156:18830:6997',
 'lh00134:653:22MKYCLT4:3:1156:18847:6997',
 'lh00134:653:22MKYCLT4:3:1171:19825:28450',
 'lh00134:653:22MKYCLT4:3:1175:24874:20379',
 'lh00134:653:22MKYCLT4:3:1187:4616:19706',
 'lh00134:653:22MKYCLT4:3:1193:12925:26390',
 'lh00134:653:22MKYCLT4:3:1206:46514:12910',
 'lh00134:653:22MKYCLT4:3:1208:12342:1448',
 'lh00134:653:22MKYCLT4:3:1210:21039:12672',
 'lh00134:653:22MKYCLT4:3:1225:51393:3844',
 'lh00134:653:22MKYCLT4:3:1228:7108:6227',
 'lh00134:653:22MKYCLT4:3:1233:40495:5316',
 'lh00134:653:22MKYCLT4:3:1254:29566:12700',
 'lh00134:653:22MKYCLT4:3:1256:37987:5540',
 'lh00134:653:22MKYCLT4:3:1271:21597:1757',
 'lh00134:653:22MKYCLT4:3

In [35]:
with open("readnames_with_well_rc_adj.txt", "w") as f:
    for item in result.tolist():
        f.write(str(item) + "\n")
